In [1]:
from adam.core.computations import KinDynComputations
from adam.core.computationsWithHardwareParameters import KinDynComputationsWithHardwareParameters
from adam.geometry import utils
import numpy as np
import casadi as cs
import gym_ignition_models

In [2]:
def SX2DM(sx):
    return cs.DM(sx)

## Import the robot .urdf

In [3]:
urdf_path = gym_ignition_models.get_model_file("iCubGazeboV2_5")
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
# Specify the root link
root_link = 'root_link'
kinDyn = KinDynComputations(urdf_path, joints_name_list, root_link)
kinDynWithHardware = KinDynComputationsWithHardwareParameters(urdf_path, joints_name_list, root_link)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
0
torso_pitch
1
torso_roll
2
torso_yaw
3
l_shoulder_pitch
4
l_shoulder_roll
5
l_shoulder_yaw
6
l_elbow
7
r_shoulder_pitch
8
r_shoulder_roll
9
r_shoulder_yaw
10
r_elbow
11
l_hip_pitch
12
l_hip_roll
13
l_hip_yaw
14
l_knee
15
l_ankle_pitch
16
l_ankle_roll
17
r_hip_pitch
18
r_hip_roll
19
r_hip_yaw
20
r_knee
21
r_ankle_pitch
22
r_ankle_roll


The class `KinDynComputations` contains some algorithms for computing kinematics and dynamics quantities.

In [4]:
# Set joints and base informations
xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_PosRPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5
masses = (np.random.rand(kinDyn.tree.N))*0.1
InertiaMatrix = (np.random.rand(3,3))*0.2

## Mass Matrix

In [5]:
M_hardware = kinDynWithHardware.mass_matrix_fun()
M = kinDyn.mass_matrix_fun()
print('Mass matrix with hardware:\n', SX2DM(M_hardware(H_b, s, masses, InertiaMatrix)))
print('Mass matrix w\o:\n', SX2DM(M(H_b, s)))

Mass matrix with hardware:
 
[[2.03203, 4.21467e-08, -1.11535e-07, 1.93101e-08, -0.028272, 0.165029, -0.0836502, 0.0339601, -0.0102784, -0.0188613, 0.0608513, 0.00187173, -0.0249998, 0.0676905, 0.011946, 0.011611, 0.00630199, 0.0209054, -0.0422467, -0.0221535, -0.0122228, -0.000685963, 0.00379076, 0.0161437, 0.0480583, 0.0125577, -0.00259273, 0.00288166, 8.53003e-05], 
 [4.21467e-08, 2.03202, 4.51936e-09, 0.028272, -1.37361e-09, 0.252951, -0.125974, 0.176477, -0.0356519, -0.00769044, -0.0109646, -0.00041134, -0.0104943, -0.00496461, 0.0148289, -0.00119041, 0.017707, 0.0152775, -0.0283471, -0.0154116, 0.0240779, 0.00224889, 0.00089331, 0.0276555, 0.0509703, 0.0178259, 0.0206672, -0.000500591, 0.000629062], 
 [-1.11535e-07, 4.51936e-09, 2.03203, -0.165029, -0.252951, -1.79365e-08, -0.195978, -0.113772, 0.00716841, 0.0245155, 0.0534419, 0.00192338, 0.00747512, -0.0109777, 0.0699395, -0.0042486, 0.0172156, 0.0204434, 0.00334408, -0.00859507, -0.00922847, -0.00279685, -0.00301922, 0.0437099

## Centroidal Momentum Matrix

In [6]:
Jcm_hardware = kinDynWithHardware.centroidal_momentum_matrix_fun()
Jcmm = kinDyn.centroidal_momentum_matrix_fun()
print('Centroidal Momentum Matrix with Hardware:\n', SX2DM(Jcm_hardware(H_b, s, masses, InertiaMatrix)))
print('Centroidal Momentum Matrix w\o Hardware: \n', SX2DM(Jcmm(H_b, s)))

Centroidal Momentum Matrix with Hardware:
 
[[2.03203, 4.21467e-08, -1.11535e-07, 1.93101e-08, -0.028272, 0.165029, -0.0836502, 0.0339601, -0.0102784, -0.0188613, 0.0608513, 0.00187173, -0.0249998, 0.0676905, 0.011946, 0.011611, 0.00630199, 0.0209054, -0.0422467, -0.0221535, -0.0122228, -0.000685963, 0.00379076, 0.0161437, 0.0480583, 0.0125577, -0.00259273, 0.00288166, 8.53003e-05], 
 [4.21467e-08, 2.03202, 4.51936e-09, 0.028272, -1.37361e-09, 0.252951, -0.125974, 0.176477, -0.0356519, -0.00769044, -0.0109646, -0.00041134, -0.0104943, -0.00496461, 0.0148289, -0.00119041, 0.017707, 0.0152775, -0.0283471, -0.0154116, 0.0240779, 0.00224889, 0.00089331, 0.0276555, 0.0509703, 0.0178259, 0.0206672, -0.000500591, 0.000629062], 
 [-1.11535e-07, 4.51936e-09, 2.03203, -0.165029, -0.252951, -1.79365e-08, -0.195978, -0.113772, 0.00716841, 0.0245155, 0.0534419, 0.00192338, 0.00747512, -0.0109777, 0.0699395, -0.0042486, 0.0172156, 0.0204434, 0.00334408, -0.00859507, -0.00922847, -0.00279685, -0.0030

## Center of mass position

In [7]:
CoM = kinDyn.CoM_position_fun()
CoM_hardware = kinDynWithHardware.CoM_position_fun()
print('Center of Mass position:\n', SX2DM(CoM(H_b, s)))
print('Center of Mass position:\n', SX2DM(CoM_hardware(H_b, s, masses)))

this is the CoM function with the mass
Center of Mass position:
 [-1.04747, 0.246362, -2.04766]
Center of Mass position:
 [nan, nan, nan]


## Total Mass

In [8]:
print('Total mass:\n', kinDyn.get_total_mass())

Total mass:
 33.06167270000001


## Jacobian

In [9]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian of the left sole:\n', SX2DM(J(H_b, s)))

Jacobian of the left sole:
 
[[1, 0, 0, 0, -0.0195197, 0.146083, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.108982, -0.168021, -0.112253, -0.0628971, -0.0186165, 0.0512249, 0, 0, 0, 0, 0, 0], 
 [0, 1, 0, 0.0195197, 0, 0.0535465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0902324, -0.153208, -0.096211, 0.130323, 0.0361322, 0.0134727, 0, 0, 0, 0, 0, 0], 
 [0, 0, 1, -0.146083, -0.0535465, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.12522, 0.0343643, -0.0718089, -0.0637916, -0.0126831, -0.0368069, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.150798, 0.233003, -0.352098, 0.841277, -0.841277, -0.437221, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.859612, -0.449014, 0.787891, 0.502995, -0.502995, 0.848592, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.488187, -0.86261, -0.505228, 0.198114, -0.198114, -0.297873, 0, 0, 0, 0, 0, 0]]


## Relative jacobian

In [10]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian between the root link and left sole:\n', SX2DM(J_r(s)))

Jacobian between the root link and left sole:
 
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.165822, -0.0686106, -0.123642, 0.00386661, 0.0052008, -0.0183881, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3.43327e-14, 0.123138, 0.0307205, -0.152654, -0.0400588, -0.0218253, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0905645, -0.181698, -0.103839, -0.0411244, -0.0134617, 0.0578427, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2.07045e-13, -0.935525, -0.0752936, 0.525905, -0.525905, 0.122145, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1.93474e-13, -0.977022, -0.2088, 0.2088, -0.94081, 0, 0, 0, 0, 0, 0], 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.353262, -0.199396, 0.824516, -0.824516, -0.316159, 0, 0, 0, 0, 0, 0]]


## Forward kinematics

In [11]:
H = kinDyn.forward_kinematics_fun('l_sole')
print('Forward kinematics to the left sole:\n', SX2DM(H(H_b, s)))

Forward kinematics to the left sole:
 
[[-0.437221, 0.794184, -0.42203, -1.07009], 
 [0.848592, 0.208879, -0.486066, 0.165777], 
 [-0.297873, -0.57065, -0.765265, -2.07391], 
 [0, 0, 0, 1]]


## Bias force term

In [12]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', SX2DM(h(H_b, s, v_b, s_dot, InertiaMatrix)))

RuntimeError: .../casadi/core/function_internal.hpp:1241: Assertion "arg.size()==n_in_" failed:
Incorrect number of inputs: Expected 4, got 5

## Coriolis term

In [ ]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', SX2DM(C(H_b, s, v_b, s_dot, InertiaMatrix)))

## Gravity term

In [ ]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', SX2DM(G(H_b, s, InertiaMatrix)))

In [ ]:
opti = cs.Opti()
Inertia_optiVar = opti.variable(3,3, 'symmetric')
masses_optVar = opti.variable(kinDyn.tree.N)
q_param = opti.parameter(kinDyn.NDoF)
H_b_param = opti.parameter(4,4)
H_b_new = np.random.rand(4,4)
H_b_new[:3, :3] = utils.R_from_RPY(rpy)
H_b_new[:3, 3] = xyz
M_f = kinDyn.mass_matrix_fun()
a = M_f(H_b_param,q_param, masses_optVar, Inertia_optiVar)
opti.minimize((a[1,1]))
opti.set_value(H_b_param, H_b_new)
opti.set_value(q_param, s)
opti.solver('ipopt')
sol = opti.solve()

print(sol.Inertia_optiVar)